# Main Regression
This file loads the data and runs the regression shown in Table 1 of the paper. 

In [1]:
data_folder = '' # path to data
figure_path = '' # path to figures folder location

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display, Latex, HTML
from stargazer.stargazer import Stargazer, LineLocation

# Read data

In [3]:
df_BG = pd.read_csv(data_folder + '23_level_BG_US_imputedquantiles_20250722.zip', index_col=0)
df_BG['STATEFP'] = df_BG['STATEFP'].astype(str).str.zfill(2)  # Ensure STATEFP is two digits
df_BG['COUNTYFP'] = df_BG['COUNTYFP'].astype(str).str.zfill(5)  # Ensure COUNTYFP is three digits
df_BG.index = df_BG.index.astype(str).str.zfill(12)  # Ensure BGFP is six digits

print('Mean size of a BG:', df_BG[df_BG['total_pop_byBG']>0]['total_pop_byBG'].mean())
print('Std:', df_BG[df_BG['total_pop_byBG']>0]['total_pop_byBG'].std())

Mean size of a BG: 1374.4094576338239
Std: 705.1132344046625


In [4]:
income_cols_BG = ['median_household_income_imputed_10k', 'av_income_10km_withoutBG_imputed_10k', 'av_income_50km_withoutBG_imputed_10k']
for col in income_cols_BG:
    df_BG[col + '_k'] = df_BG[col] / 1000.0  # Convert income to thousands of USD
    df_BG[col + '_10k'] = df_BG[col] / 10000.0  # Convert income to 10s of thousands of USD

# Functions

In [5]:
# Run a single regression
def run_regression(df_data,dep_var,indep_var,covs=[],state_FE=False,which='OLS'):
    df_data_noNaN = df_data.dropna(subset=[indep_var,dep_var] + covs)
    print(dep_var + ' ~ ' + indep_var + ' + '+ str(covs) + ' + ' + str(state_FE))
    X = df_data_noNaN[[indep_var] + covs]  # Independent variable(s)
    if state_FE:
        assert type(df_data_noNaN['STATEFP'].iloc[0]) == str  # Ensure STATEFP is a string
        dummies = pd.get_dummies(df_data_noNaN['STATEFP'], dtype=int, drop_first=True)
        X = pd.concat([X, dummies], axis=1)
    X = sm.add_constant(X)  # Adds a constant term to the predictor
    Y = df_data_noNaN[dep_var]  # Dependent variable
    if which == 'OLS':
        model = sm.OLS(Y, X).fit()
    elif which == 'Logit':
        model = sm.Logit(Y, X).fit()
    return model

# Table 1: Main regression

In [6]:
# Regressions for BG-level data
OLS_StateFE_imputed = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',[],state_FE='STATEFP',which='OLS')
OLS_BG_10_imputed = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_50_imputed = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_50km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')

no_stations ~ median_household_income_imputed_10k + [] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_50km_withoutBG_imputed_10k'] + STATEFP


In [7]:
list_covs_cleaned = ['median_household_income_imputed_10k', 
                     'av_income_10km_withoutBG_imputed_10k', 
                     'av_income_50km_withoutBG_imputed_10k']

dict_rename_covs = {'median_household_income_imputed_10k': 'BG median household income [10kUSD]',
                    'av_income_10km_withoutBG_imputed_10k':'Average neighbouring income [10kUSD], BG/10km',
                    'av_income_50km_withoutBG_imputed_10k':'Average neighbouring income [10kUSD], BG/50km'}

In [8]:
stargazer = Stargazer([OLS_StateFE_imputed, OLS_BG_10_imputed, OLS_BG_50_imputed])
stargazer.significance_levels([0.05, 0.01, 0.001])
stargazer.dependent_variable_name('Number of stations')
stargazer.covariate_order(list_covs_cleaned)
stargazer.rename_covariates(dict_rename_covs)

stargazer.significant_digits(3)
stargazer.show_degrees_of_freedom(False)

stargazer

In [9]:
htmlstring = stargazer.render_html()
with open(figure_path + "Figures/Table_1.html", "w") as file:
    file.write(htmlstring)
    
# Generate LaTeX output
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{3}{c}{\textit{Dependent variable: Number of stations}} \
\cr \cline{2-4}
\\[-1.8ex] & (1) & (2) & (3) \\
\hline \\[-1.8ex]
 BG median household income [10kUSD] & 0.003$^{***}$ & -0.012$^{***}$ & -0.003$^{***}$ \\
& (0.001) & (0.001) & (0.001) \\
 Average neighbouring income [10kUSD], BG/10km & & 0.056$^{***}$ & \\
& & (0.002) & \\
 Average neighbouring income [10kUSD], BG/50km & & & 0.054$^{***}$ \\
& & & (0.003) \\
\hline \\[-1.8ex]
 Observations & 239780 & 239766 & 239776 \\
 $R^2$ & 0.010 & 0.013 & 0.012 \\
 Adjusted $R^2$ & 0.010 & 0.013 & 0.012 \\
 Residual Std. Error & 1.643 & 1.640 & 1.641 \\
 F Statistic & 48.536$^{***}$ & 62.087$^{***}$ & 55.224$^{***}$ \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \multicolumn{3}{r}{$^{*}$p$<$0.05; $^{**}$p$<$0.01; $^{***}$p$<$0.001} \\
\end{tabular}
\end{table}
